#초기 환경 설정

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요 패키지 설치
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch>=1.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292697 sha256=03aa8e1206ba60eed98a40b3dd807030662aa873d10c5b778b8770758be85cb7
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.9 MB/s eta 0:00:00


In [ ]:
# Kobert 실행
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-hothe5r4/kobert-tokenizer_0df9b224445d402896059c96fe11f185
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-hothe5r4/kobert-tokenizer_0df9b224445d402896059c96fe11f185
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=a39e9bac682a48cf526020f931360665272e42b770e198f4cabe7de2c1e8f60d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3thu30jq/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
# 필요 패키지 import 1
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import json

# transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# 필요 패키지 import 2
!pip install xmltodict

import xmltodict
import requests

In [ ]:
# no mouduled kobert 문제 해결
# ★ Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
# 모델 객체 생성
# 토크나이저와 bertmodel 우회 생성
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

# 리뷰 데이터 작업

In [ ]:
import pandas as pd
import os
import glob
import numpy as np

In [ ]:
place_list = pd.read_excel('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/관광지 목록 수정본.xlsx')
place_list.drop(['NO'], axis=1, inplace=True)

#전체 파일 불러오기
def load_csv_files_naver(dir):
    files = os.listdir(dir)
    csv_files = [file for file in files if file.endswith('.csv')] #파일 이름 불러오기

    dataframes = {}
    for file in csv_files:
        dataframe_name = os.path.splitext(file)[0]  # 파일 이름에서 확장자 제거
        data = pd.read_csv(os.path.join(dir, file))
        data = list(data.iloc[:,1])
        dataframes[dataframe_name] = data

    return dataframes

def load_csv_files_trip(dir):
    files = os.listdir(dir)
    csv_files = [file for file in files if file.endswith('.csv')] #파일 이름 불러오기

    dataframes = {}
    for file in csv_files:
        dataframe_name = os.path.splitext(file)[0]  # 파일 이름에서 확장자 제거
        data = pd.read_csv(os.path.join(dir, file))
        data = list(data.iloc[:,4])
        dataframes[dataframe_name] = data

    return dataframes


dir_naver = '/content/drive/MyDrive/DScover/2023-2 메인프/데이터/리뷰 area code명/네이버 리뷰'
dir_trip = '/content/drive/MyDrive/DScover/2023-2 메인프/데이터/리뷰 area code명/트립어드바이저 크롤링'
result_naver = load_csv_files_naver(dir_naver)
result_trip = load_csv_files_trip(dir_trip)

In [ ]:
merged_dict = {**result_naver, **result_trip}

In [ ]:
merged_dict.keys()

In [ ]:
# key값 중 ddp 키값 수정
merged_dict['POI083'] = merged_dict.pop('동대문디자인플라자(트립이랑 합치기)')

In [ ]:
# 다시 key값 체크 > 맞음
merged_dict.keys()

dict_keys(['POI032', 'POI110', 'POI111', 'POI112', 'POI108', 'POI113', 'POI109', 'POI103', 'POI105', 'POI102', 'POI101', 'POI099', 'POI091', 'POI011', 'POI087', 'POI095', 'POI094', 'POI096', 'POI098-2', 'POI097', 'POI093', 'POI098', 'POI092', 'POI008', 'POI012-2', 'POI009', 'POI001-2', 'POI001', 'POI002', 'POI003', 'POI004', 'POI005', 'POI005-3', 'POI005-2', 'POI005-4', 'POI060', 'POI055', 'POI009-2', 'POI010', 'POI012', 'POI017', 'POI034', 'POI038', 'POI040', 'POI045', 'POI046', 'POI054', 'POI057', 'POI058', 'POI059', 'POI062', 'POI063', 'POI064', 'POI066', 'POI067', 'POI073', 'POI074', 'POI078', 'POI081', 'POI082', 'POI083', 'POI084', 'POI086', 'POI088', 'POI090', 'POI104', 'POI100', 'POI014'])

In [ ]:
merged_dict.values()

In [ ]:
# 데이터 예시 보기 : key값이 "POI102" 일때
merged_dict['POI102']

### 리뷰 데이터 딕셔너리 생성

In [ ]:
# 각 관광지 ID별 리뷰 데이터프레임을 저장할 딕셔너리
df_dict = {}

# merged_dict 내의 각 관광지 ID와 리뷰에 대한 처리
for key, reviews in merged_dict.items():
    # 리스트를 DataFrame으로 변환하며, 열 이름을 'content'로 지정
    reviews_df = pd.DataFrame(reviews, columns=['content'])

    # content 열을 문자열로 변환
    reviews_df['content'] = reviews_df['content'].astype(str)

    # 중복 제거, 한글 처리, 띄어쓰기 조정
    reviews_df.drop_duplicates(inplace=True)
    reviews_df['content'] = reviews_df['content'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
    reviews_df['content'] = reviews_df['content'].str.replace(r"\s+", " ", regex=True)  # 연속된 공백을 하나의 공백으로 대체
    reviews_df['content'] = reviews_df['content'].str.strip()  # 문자열 앞뒤의 공백 제거

    # 빈 문자열 제거
    reviews_df['content'].replace('', pd.NA, inplace=True)  # 빈 문자열을 NA 값으로 변경
    reviews_df.dropna(inplace=True)  # NA 값이 있는 행 제거

    # 전처리된 데이터프레임을 df_dict에 저장
    df_dict[key] = reviews_df

In [ ]:
# 예시: POI102에 대한 데이터프레임 확인
print(df_dict['POI102'].head())

In [ ]:
# 리뷰 수 전처리를 위한 boxplot 설정
import matplotlib.pyplot as plt

# df_dict에서 각 데이터프레임의 행 수를 리스트로 추출
rows_df = [len(df) for df in df_dict.values()]

# 박스플롯 생성
plt.figure(figsize=(10, 6))
plt.boxplot(rows_df)
plt.title("리뷰들의 개수 분포")
plt.ylabel("리뷰개수")
plt.show()

In [ ]:
# 통계값 수치로 표현
median = np.median(rows_df)  # 중앙값
mean = np.mean(rows_df)      # 평균
q1 = np.percentile(rows_df, 25)  # 제1사분위수 (Q1)
q3 = np.percentile(rows_df, 75)  # 제3사분위수 (Q3)

print(median, mean, q1, q3)

# 평균인 290개의 리뷰만 사용하기로 결정

106.0 290.52941176470586 31.75 288.25


In [ ]:
# 리뷰 수 줄이는 전처리
# 임베딩에 사용할 최종 딕셔너리 생성
final_dict = {}

for key, reviews_df in df_dict.items():
    if len(reviews_df) <= 290:
        # 데이터프레임의 크기가 290 이하인 경우 모든 행을 사용
        final_dict[key] = reviews_df
    else:
        # 데이터프레임의 크기가 290보다 큰 경우 무작위로 290개의 행을 추출
        final_dict[key] = reviews_df.sample(n=290, random_state=1234)

In [ ]:
final_dict

In [ ]:
# final_dict를 json 파일로 저장
import json

# 데이터프레임을 JSON 형식으로 변환하여 final_dict에 저장
final_dict_json = {key: df.to_json(orient='records') for key, df in final_dict.items()}

# final_dict_json을 JSON 파일로 저장
with open('final_dict.json', 'w', encoding='utf-8') as f:
    json.dump(final_dict_json, f, ensure_ascii=False)

### 리뷰 데이터 딕셔너리에 kobert 모델 적용해 임베딩 데이터 생성

In [ ]:
# JSON 파일 로드
with open('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/final_dict.json', 'r', encoding='utf-8') as f:
    loaded_dict_json = json.load(f)

# JSON을 다시 데이터프레임으로 변환
loaded_dict = {key: pd.read_json(json_str, orient='records') for key, json_str in loaded_dict_json.items()}

In [ ]:
# 데이터 구조 확인
loaded_dict

In [ ]:
# 임베딩 함수 설정
# 처리 속도에 따라 max_length와 batch_size 조정 중
def create_kobert_embeddings(reviews_df, tokenizer, bertmodel, max_length=512, batch_size=32):
    # GPU 사용 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    bertmodel.to(device)

    # 토큰화 및 정수 인덱스 변환 함수
    def tokenize_data(text):
        return tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            truncation=True,
            padding='max_length',
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt'
        )

    # DataLoader 설정
    data_loader = DataLoader(reviews_df['content'].tolist(), batch_size=batch_size)  # 수정된 부분

    # 임베딩 벡터를 저장할 리스트
    embeddings = []

    # 데이터 로더를 통해 배치 단위로 임베딩 생성
    bertmodel.eval()
    with torch.no_grad():
        for texts in data_loader:
            inputs = [tokenize_data(text) for text in texts]
            input_ids = torch.cat([x['input_ids'] for x in inputs], dim=0).to(device)
            attention_mask = torch.cat([x['attention_mask'] for x in inputs], dim=0).to(device)

            outputs = bertmodel(input_ids, attention_mask=attention_mask)
            batch_embeddings = outputs[0].mean(dim=1).cpu().numpy()  # CLS 토큰의 임베딩 사용
            embeddings.extend(batch_embeddings)

    return np.array(embeddings)

In [ ]:
# 각 관광지별 리뷰 데이터에 대한 임베딩 생성
# 경과 속도를 알아볼 tdqm 라이브러리 import
from tqdm import tqdm

# 임베딩 결과값 담을 딕셔너리 생성
embeddings_dict = {}

# 리뷰값이 비어있는 관광지(POI104) 제거
del loaded_dict['POI014']

# 임베딩 생성 반복문 실행
for key in tqdm(loaded_dict.keys(), desc="Processing"):
    reviews_df = loaded_dict[key]
    embeddings = create_kobert_embeddings(reviews_df, tokenizer, bertmodel)
    embeddings_dict[key] = embeddings

Processing: 100%|██████████| 67/67 [05:06<00:00,  4.58s/it]


In [ ]:
# 예시 : POI102에 대한 임베딩결과 확인
print(embeddings_dict['POI102'])

[[ 0.08602966 -0.35273752 -0.12926638 ... -0.11971171 -0.22522311
   0.10809118]
 [ 0.282117   -0.4027114   0.19481839 ... -0.27549416 -0.11033238
   0.03138701]
 [ 0.01652298 -0.246156    0.1363411  ... -0.0901429  -0.34894508
   0.07066087]
 ...
 [ 0.16724809 -0.33610344 -0.06946187 ... -0.25019246 -0.21871768
   0.21711633]
 [ 0.32292023 -0.41144797 -0.04827482 ... -0.01992172 -0.16827008
   0.23612022]
 [ 0.3087889  -0.3713088  -0.0784919  ... -0.33274195 -0.2826385
  -0.25335154]]


In [ ]:
# 임베딩 벡터 json 파일로 저장

# 임베딩 벡터를 리스트로 변환하여 JSON 저장 가능한 형태로 만듦
embeddings_dict_json = {key: embeddings.tolist() for key, embeddings in embeddings_dict.items()}

# JSON 파일로 저장
with open('embeddings_dict_reviews.json', 'w', encoding='utf-8') as f:
    json.dump(embeddings_dict_json, f, ensure_ascii=False)

In [ ]:
# 런타임 재배정 시, 임베딩값 파일 불러오기
import json

with open('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/embeddings_dict_reviews.json', 'r', encoding='utf-8') as f:
    loaded_embeddings_dict_json = json.load(f)

# 리스트를 다시 Numpy 배열로 변환
loaded_embeddings_dict = {key: np.array(embeddings) for key, embeddings in loaded_embeddings_dict_json.items()}

In [ ]:
# 데이터 불러오기가 적절한지 확인
loaded_embeddings_dict

# 실시간 데이터 작업

In [ ]:
# 장소 리스트 불러오기
lst = pd.read_excel("/content/drive/MyDrive/DScover/2023-2 메인프/데이터/서울시 주요 113장소명 목록 (91개 수정본).xlsx")

# 각 관광지에 대한 실시간 데이터 주소
space_num = []
area_name = []
for i in range(len(lst)):
    space_num.append(str(lst.at[i,'NO']))
    area_name.append(str(lst.at[i,'AREA_NM']))
web_address = []
address_str = "http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/"

for i in range(len(lst)):
    address = address_str + space_num[i] + '/' + area_name[i]
    web_address.append(address)

web_address

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


['http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/1/강남 MICE 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/2/동대문 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/3/명동 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/4/이태원 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/5/잠실 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/6/종로·청계 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/7/홍대 관광특구',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/8/경복궁',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/9/광화문·덕수궁',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata/1/10/보신각',
 'http://openapi.seoul.go.kr:8088/554763564a65686438314c744c4351/xml/citydata

In [ ]:
# 전체 실시간 데이터 수집
key = '6b6c58426167726b3439424b4d7873'

#전체 데이터가 담긴 리스트 생성
all_data = []

for i in range(len(lst)):
    response = requests.get(web_address[i])
    data = xmltodict.parse(response.text)

    # XML 데이터를 JSON 형식으로 변환
    json_data = json.loads(json.dumps(data))

    # JSON 데이터를 데이터프레임으로 변환
    df = pd.json_normalize(data)

    all_data.append(df)

In [ ]:
# 실시간 데이터프레임의 컬럼명과 한글 이름 데이터
column_list = pd.read_excel('/content/drive/MyDrive/DScover/2023-2 메인프/데이터/컬럼명 리스트.xlsx')
column_list = column_list.drop(['No'], axis = 1)

In [ ]:
import copy
# 원본 복사 (데이터 훼손 방지)
all_data2 = copy.deepcopy(all_data)

# all_data의 각 장소 정보를 하나의 데이터 프레임으로
merged_df = pd.DataFrame()
for i in all_data2:
    merged_df = pd.concat([merged_df, i], ignore_index=True)

# 데이터프레임 컬럼명 일부 한글화
merged_df = merged_df.drop(['SeoulRtd.citydata.list_total_count',
       'SeoulRtd.citydata.RESULT.RESULT.CODE',
       'SeoulRtd.citydata.RESULT.RESULT.MESSAGE'], axis=1)
merged_df.rename(columns={'SeoulRtd.citydata.CITYDATA.ROAD_TRAFFIC_STTS.AVG_ROAD_DATA.ROAD_TRFFIC_TIME' : '도로소통현황 업데이트 시간',
                            'SeoulRtd.citydata.CITYDATA.EVENT_STTS.EVENT_STTS' : '행사 이름'}, inplace=True)

In [ ]:
#컬럼명 전체 한글화
duplicated_columns = merged_df.columns
for j in range(len(duplicated_columns)):
    for i in range(len(column_list)):
        if merged_df.columns[j].endswith(column_list.iat[i,0]):
            merged_df.rename(columns={merged_df.columns[j]: column_list.iat[i, 1]}, inplace=True)
            break

In [ ]:
#문화 행사 정보 필터
class event_filter:
    def __init__(self, df):
        self.df = df

    #문화 행사가 존재하는 곳에 대한 필터링
    def event(self):
        event_df = self.df.loc[(self.df['문화행사명'].isna() == False) | (self.df['행사 이름'].isna() == False)].reset_index(drop=True)
        return event_df

    def __del__(self):
        pass

#주차장 정보 필터
class parkinglot_filter:
    def __init__(self, df):
        self.df = df

    def parking_lot(self):
        parking_lot_df = self.df.loc[(self.df['주차장명'].isna() == False)].reset_index(drop=True)

        return parking_lot_df

    def __del__(self):
        pass

class complexity_filter:
    def __init__(self,df):
        self.df = df

# 장소 혼잡도 메세지에 따라 필터링
    def complexity(self):
        complexity_df = self.df.loc[(self.df['장소 혼잡도 지표 관련 메세지'] == '사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.')
        | (self.df['장소 혼잡도 지표 관련 메세지'] == '사람이 몰려있을 수 있지만 크게 붐비지는 않아요. 도보 이동에 큰 제약이 없어요.')].reset_index(drop=True)

        return complexity_df

In [ ]:
# 필터링 후 정보 출력 : print_info() 클래스 설정

class print_info:
    def __init__(self, df, option):
        self.df = df
        self.option = option

    def display_info(self):
        for i in range(len(self.df)):
            print("핫스팟 장소명: {}".format(self.df.iloc[i]['핫스팟 장소명']))

            # 날씨 정보 프린트
            print('날씨 정보:')
            print('\t강수관련정보: {}'.format(self.df.iloc[i]['강수관련 메세지']))
            print('\t자외선메세지: {}'.format(self.df.iloc[i]['자외선메세지']))
            print('\t미세먼지/초미세먼지 지표: {}'.format(self.df.iloc[i]['미세먼지지표'], self.df.iloc[i]['초미세먼지지표']))

            # 문화 행사 출력
            if self.option[0] == 1:
                # 문화행사명이 NaN이 아닌 경우에만 출력
                if pd.notna(self.df.iloc[i]['문화행사명']):
                    print('\t문화행사명: {}'.format(self.df.iloc[i]['문화행사명']))
                # 행사 이름이 리스트 형태인 경우에만 반복
                if '행사 이름' in self.df.columns and isinstance(self.df.iloc[i]['행사 이름'], list):
                    for event in self.df.iloc[i]['행사 이름']:
                        print('\t문화행사명 및 장소: {}\t(위치: {})'.format(event['EVENT_NM'], event['EVENT_PLACE']))

            # 주차장 정보 출력
            if self.option[1] == 1 and pd.notna(self.df.iloc[i]['주차장명']):
                print('\n\t주차장 정보:')
                print('\t주차장명: {}'.format(self.df.iloc[i]['주차장명']))
                print('\t현재수용가능명: {}'.format(self.df.iloc[i]['주차장 수용 가능 면수']))
                print('\t유무료 여부: {}'.format(self.df.iloc[i]['유무료 여부']))

            # 혼잡도 정보 출력
            if self.option[2] == 1 and pd.notna(self.df.iloc[i]['장소 혼잡도 지표 관련 메세지']):
                print("\n\t혼잡도 정보: {}".format(self.df.iloc[i]['장소 혼잡도 지표 관련 메세지']))

            print('\n')

In [ ]:
class execute_filtering:

    def __init__(self, df, options):
        self.df = df
        self.options = options

    def filtering(self):
        # 데이터 필터링 전 원본 복사
        filtered_df = copy.deepcopy(self.df)

        for option in range(len(self.options)):
            if self.options[option] == 1 :

                #행사 정보 필터링
                if option == 0:
                    filtered_df = event_filter(filtered_df).event()

                # 주차장 정보 기반 필터링
                elif option == 1:
                    filtered_df = parkinglot_filter(filtered_df).parking_lot()

                # 혼잡도 정보 기반 필터링
                elif option == 2:
                    filtered_df = complexity_filter(filtered_df).complexity()

        return filtered_df

In [ ]:
# 필터링 조건 설정 및 결과 테스트
event_opt = 1
parkinglot_opt = 1
complexity_opt = 1

options = [event_opt, parkinglot_opt, complexity_opt]

result = execute_filtering(merged_df, options)
filtered_result = result.filtering()

# 필터링된 리뷰 딕셔너리와 사용자 선호 키워드간의 유사도 도출 및 관광지 추천

#####  실시간 데이터 중 사용자의 의도에 맞는 관광지만 추출

In [ ]:
# 사용자에게 원하는 조건을 물어보는 질문
# 조건 설정
conditions = ["행사를 하는 관광지에 가고 싶어요",
              "주차장이 필요해요",
              "사람 많은 곳은 싫어요"]

# 조건 받기
selected_condition = []
print("다음 중 원하는 조건을 선택하세요 (예: 1, 2, 3, 0을 입력시 종료됩니다):")
for i, condition in enumerate(conditions, start=1):
    print(f"{i}. {condition}")

# 사용자 입력 받고, 받은 응답 수정
choice_input = input("숫자를 입력하여 조건을 선택하세요: ")
choices = choice_input.replace(" ", "").split(',')

# 입력된 선택 처리
for choice in choices:
    choice = int(choice)
    selected_condition.append(conditions[choice - 1])

# 선택된 조건 출력
print("선택된 조건:", selected_condition)

다음 중 원하는 조건을 선택하세요 (예: 1, 2, 3, 0을 입력시 종료됩니다):
1. 행사를 하는 관광지에 가고 싶어요
2. 주차장이 필요해요
3. 사람 많은 곳은 싫어요
숫자를 입력하여 조건을 선택하세요: 1, 3
선택된 조건: ['행사를 하는 관광지에 가고 싶어요', '사람 많은 곳은 싫어요']


In [ ]:
# 조건에 맞는 옵션 설정
options = [0, 0, 0]  # 기본값은 모든 옵션 비활성화

for condition in selected_condition:
    if condition == "행사를 하는 관광지에 가고 싶어요":
        options[0] = 1
    elif condition == "주차장이 필요해요":
        options[1] = 1
    elif condition == "사람 많은 곳은 싫어요":
        options[2] = 1

# 필터링 실행
result = execute_filtering(merged_df, options)
filtered_result = result.filtering()

# 필터링된 결과 출력
filtered_result

In [ ]:
# 필터링된 데이터의 핫스팟 코드명 추출
filtered_placeID = filtered_result['핫스팟 코드명']

In [ ]:
# 조건에 맞는 관광지들만의 리뷰 임베딩값 딕셔너리  생성
selected_dict = {key: loaded_embeddings_dict[key] for key in filtered_placeID if key in loaded_embeddings_dict}

# 결과 확인
print(selected_dict)

{'POI001': array([[ 1.45721599e-01, -3.73294875e-02, -1.91380382e-01, ...,
        -3.93926471e-01, -2.23549545e-01, -4.22848796e-04],
       [ 1.29348025e-01, -4.36095536e-01, -1.57475710e-01, ...,
        -8.93478394e-02, -4.29413561e-03,  1.74821839e-01],
       [ 5.32891154e-02, -3.45733970e-01, -1.22964822e-01, ...,
        -1.97266579e-01, -2.54840016e-01,  1.26396850e-01],
       ...,
       [-3.23207527e-02, -4.79690880e-01, -1.62613802e-02, ...,
        -1.84948325e-01,  2.09826902e-02,  3.05025578e-01],
       [ 1.20877363e-01, -2.58114129e-01, -2.17480063e-01, ...,
        -1.16484299e-01, -2.23006949e-01,  1.50323838e-01],
       [ 2.29334965e-01, -3.17899704e-01,  2.14851014e-02, ...,
        -1.89513266e-01, -1.66465238e-01,  4.88835294e-03]]), 'POI002': array([[-0.06786932, -0.10019659,  0.04612298, ..., -0.18329933,
        -0.16512625,  0.11733081],
       [ 0.08568452, -0.29952157,  0.24036533, ..., -0.07373932,
        -0.08705314,  0.1333881 ],
       [ 0.16314612, 

In [ ]:
# 조건에 맞는 관광지만 추출되었는지 확인
selected_dict.keys()

dict_keys(['POI001', 'POI002', 'POI003', 'POI004', 'POI005', 'POI008', 'POI009', 'POI032', 'POI040', 'POI054', 'POI064', 'POI066', 'POI067', 'POI078', 'POI083', 'POI086', 'POI088', 'POI091', 'POI092', 'POI096', 'POI097', 'POI104', 'POI111'])

### 리뷰와 키워드간의 유사도 도출 코드


### 추천 결과 확인

In [ ]:
# 키워드값 예시 입력
keywords = ["사진 찍기 좋아요.", "볼거리가 많아요.", "산책로가 잘 되어있어요.",
            "아이와 가기 좋아요.",  "체험 프로그램이 다양해요.", "반려동물과 가기 좋아요.",
            "뷰가 좋아요.", "관리가 잘 되어있어요.", "맛집이 많아요."]

In [ ]:
# 사용자에게 선호 키워드를 입력받기
print("다음 중 원하는 키워드를 선택하세요 (0을 입력시 종료됩니다):")
for i, keyword in enumerate(keywords, start=1):
    print(f"{i}. {keyword}")

# 키워드를 받을 리스트 생성
selected_keyword = []

# 사용자 선택 받기
for i in range(1, len(keywords) + 1) :
  choice = int(input(f"숫자를 입력하여 {i}번째 키워드를 선택하세요: "))
  # 0을 입력시 종료
  if choice == 0 :
    break

  selected_keyword.append(keywords[choice - 1])

print("선택한 키워드:", selected_keyword)

다음 중 원하는 키워드를 선택하세요 (0을 입력시 종료됩니다):
1. 사진 찍기 좋아요.
2. 볼거리가 많아요.
3. 산책로가 잘 되어있어요.
4. 아이와 가기 좋아요.
5. 체험 프로그램이 다양해요.
6. 반려동물과 가기 좋아요.
7. 뷰가 좋아요.
8. 관리가 잘 되어있어요.
9. 맛집이 많아요.
숫자를 입력하여 1번째 키워드를 선택하세요: 4
숫자를 입력하여 2번째 키워드를 선택하세요: 6
숫자를 입력하여 3번째 키워드를 선택하세요: 8
숫자를 입력하여 4번째 키워드를 선택하세요: 9
숫자를 입력하여 5번째 키워드를 선택하세요: 0
선택한 키워드: ['아이와 가기 좋아요.', '반려동물과 가기 좋아요.', '관리가 잘 되어있어요.', '맛집이 많아요.']


In [ ]:
# 입력받은 키워드값과 리뷰간의 코사인 유사도 측정 및 추천 관광지 도출 함수
from sklearn.metrics.pairwise import cosine_similarity

def recommend_attractions_by_keywords(keywords, embeddings_dict, tokenizer, bertmodel):
    # 키워드를 임베딩으로 변환
    keywords_embeddings = create_kobert_embeddings(pd.DataFrame({'content': keywords}), tokenizer, bertmodel)

    # 코사인 유사도 계산 및 결과 저장
    recommendations = {}
    for attraction, embeddings in embeddings_dict.items():
        # 각 관광지에 대한 평균 유사도 계산
        similarity_scores = [cosine_similarity([keyword_embedding], embeddings).mean() for keyword_embedding in keywords_embeddings]
        average_similarity = np.mean(similarity_scores)
        recommendations[attraction] = average_similarity

    # 평균 유사도에 따라 관광지 정렬
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    return sorted_recommendations

In [ ]:
# 함수 적용 1번
# 선택한 키워드: ['산책로가 잘 되어있어요.', '반려동물과 가기 좋아요.', '뷰가 좋아요.', '사진 찍기 좋아요.']
# 결과 : 서촌, 어린이대공원, 고척돔

recommended_attractions = recommend_attractions_by_keywords(selected_keyword, selected_dict, tokenizer, bertmodel)
print(recommended_attractions[:3]) # 상위 3개 추천 관광지

[('POI067', 0.6753388893418981), ('POI104', 0.6725917136557213), ('POI086', 0.6666967303208066)]


In [ ]:
# 함수 적용 2번
# 선택한 키워드: ['아이와 가기 좋아요.', '반려동물과 가기 좋아요.', '관리가 잘 되어있어요.', '맛집이 많아요.']
# 결과 : 서촌, 어린이대공원, 북촌한옥마을

recommended_attractions = recommend_attractions_by_keywords(selected_keyword, selected_dict, tokenizer, bertmodel)
print(recommended_attractions[:3]) # 상위 3개 추천 관광지

[('POI067', 0.6827559386219854), ('POI104', 0.6779699564369243), ('POI066', 0.6649157581361604)]


In [ ]:
# 추천된 관광지 ID 추출
recommended_ids = [item[0] for item in recommended_attractions[:3]]

# 추천된 관광지에 대한 정보를 포함하는 데이터프레임 생성
recommended_df = merged_df[merged_df['핫스팟 코드명'].isin(recommended_ids)]

# 추천된 관광지 정보 출력
print_info(recommended_df, options).display_info()


핫스팟 장소명: 북촌한옥마을
날씨 정보:
	강수관련정보: 비 또는 눈 소식이 없어요.
	자외선메세지: 햇볕에 민감한 분들은 자외선 차단제를 발라주세요.
	미세먼지/초미세먼지 지표: 좋음
	문화행사명 및 장소: [서울공예박물관] 2023 KZ 프로젝트 [만년사물]	(위치: 서울공예박물관 전시1동 3층)
	문화행사명 및 장소: [서울공예박물관] 시민소통 공예프로그램 공예@쇼윈도 #6 [옹기와 숨ː]	(위치: 서울공예박물관 전시3동 1층 쇼윈도)
	문화행사명 및 장소: 2023 송현동 솔빛축제	(위치: 열린송현 녹지광장)
	문화행사명 및 장소: 고희동미술관 재개관 3주년기념전시 「모던을 터치하다: 고희동， 근대회화의 선구자」	(위치: 고희동미술관 (종로구 창덕궁5길 40))

	혼잡도 정보: 사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.


핫스팟 장소명: 서촌
날씨 정보:
	강수관련정보: 비 또는 눈 소식이 없어요.
	자외선메세지: 햇볕에 민감한 분들은 자외선 차단제를 발라주세요.
	미세먼지/초미세먼지 지표: 좋음
	문화행사명: 박노수미술관 개관 10주년 기념전시 [화필인생-박노수의 모던 타임즈]

	혼잡도 정보: 사람이 몰려있을 가능성이 낮고 붐빔은 거의 느껴지지 않아요. 도보 이동이 자유로워요.


핫스팟 장소명: 어린이대공원
날씨 정보:
	강수관련정보: 비 또는 눈 소식이 없어요.
	자외선메세지: 햇볕에 민감한 분들은 자외선 차단제를 발라주세요.
	미세먼지/초미세먼지 지표: 좋음
	문화행사명 및 장소: [서울상상나라] 눈사람 크림빵	(위치: 서울상상나라 지하 1층 요리놀이실)
	문화행사명 및 장소: [서울상상나라] 루돌프 브라우니	(위치: 서울상상나라 지하 1층 요리놀이실)
	문화행사명 및 장소: [서울상상나라] 우리， 캠핑 가요!	(위치: 서울상상나라 3층 문화놀이)
	문화행사명 및 장소: [서울상상나라] 불멍별멍! 캠핑 놀이	(위치: 서울상상나라 지하 1층 표현놀이실)

	혼잡도 정보: 사람이 몰려있을 가능성이 낮고